In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores.faiss import FAISS

In [2]:
with open('../state_of_the_union.txt') as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

embeddings = CohereEmbeddings()

In [3]:
docsearch = FAISS.from_texts(texts, embeddings)

In [4]:
# Add in a fake source information
for i, d in enumerate(docsearch.docstore._dict.values()):
    d.metadata = {'source': f"{i}-pl"}

In [5]:
query = "What did the president say about Justice Breyer"
docs = docsearch.similarity_search(query)

In [6]:
from langchain.chains.qa_with_sources.base import QAWithSourcesChain
from langchain.llms import OpenAI, Cohere
from langchain.docstore.document import Document

In [7]:
chain = QAWithSourcesChain.from_llm(OpenAI(temperature=0))

In [8]:
chain({"docs": docs, "question": query}, )

{'docs': [Document(page_content='Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence. \n\nA former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since she’s been nominated, she’s received a broad range of support—from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. \n\nAnd if we are to advance liberty and justice,

In [9]:
from langchain.chains.qa_with_sources.vector_db import VectorDBQAWithSourcesChain

In [10]:
chain = VectorDBQAWithSourcesChain.from_llm(OpenAI(temperature=0), vectorstore=docsearch)

In [11]:
chain({"question": "What did the president say about Justice Breyer"}, return_only_outputs=True)

{'answer': ' The president said that Justice Breyer is an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. ',
 'sources': '27-pl'}